# Extranting data from OCR JSON
 

In the following notebook, I is show how to extract specific information of two specific OCR .json files. 
The information is presented as JSON format with the following structure:

{'Date' : value,
 'Store address' : value,
 'Invoice number' : value,
 'Subtotal' : value,
 'Total' : value,
 'Line items': values}

Because two differents files are provided, I create severals functions for each one and funtions that can be executed for both files.

## Import the necessary python libraries

In [6]:
import numpy as np
import json

## Load the .json file

 The path './OCR_ticket1.json' is the path for the provided samples. Change it to the path of your .json path.
 
 <p>ticket1 refers to JUMBO CALLE 80</p>
 <p>ticket2 refers to JUMBO VALLE DE LILI</p>

In [7]:
# Open the JSON files and load the JSON data 

with open('./OCR_ticket1.json', 'r') as file:
    
    ticket1 = json.load(file) 

with open('./OCR_ticket2.json', 'r') as file:
    
    ticket2 = json.load(file)

<p> Note: After looking the .json for both images, I concluded that the hole and relevant information was in one place: </p>
<p> file_json['pages'][0]['textAnnotations'][0]['description'] </p>


## Function to extract the Shop adress



In [8]:
'''
This function read the shop adress. The .json files provided are the result of OCR process of two sell tickets 
of wo differents shops: JUMBO CALLE 80 and JUMBO VALLE DE LILI. 
So, this function only read the next forward string of the shop name.  

'''

def adress(b):
    
    c = []
    
    lent = np.arange(len(b))
    
    for i in lent:
        if b[i] == 'JUMBO VALLE DE LILI':
            c.append(b[i+1])
        elif b[i] == 'JUMBO CALLE 80':
            c.append(b[i+1])
    return c

## Function to extrac the items of one .json

This function only works for .json associated with CALLE 80 shop

In [9]:
'''

 Due to the structure of the file, it was necessary to identify the place 
 where the required information of the sold items appeared. This function 
 is responsible for finding the places where the required information is 
 found and cleans the data for better manipulation.
 
'''

def items_calle80(text_split): 
    
    tex = np.arange(len(text_split)) 
    
    # Set the integers to use 
    
    start = 0
    end = 0
    
    # Set the lists to use
    
    sku = []
    desc = []
    val = []
    tax_ty = []
    data_1 = []
    data_2 = []
    items = []
    
    # Determines where the block of interest begins and ends
    
    for i in tex:
        if text_split[i] == '$': 
            start = i
        if text_split[i] == 'SUBTOTAL.': 
            end = i
    
    # Extract the necesary data
    
    for i in np.arange(start+1,end,1):
        data_1.append(text_split[i])      
    
    # This is necessary to remove unnecessary data
    
    data_1.remove('SUBTOTAL')
    
    # Split the data into rows
    
    for i in data_1:
        data_2.append(i.split(' ',1))
    
    # Clean the data and create the atributes of the sell items in lits
    
    for i in data_2:
        if (len(i) == 2):
            if (len(i[0]) <= 8 and i[0].isnumeric() == True):
                val.append(i[0])
                tax_ty.append(i[1])
            else: 
                sku.append(i[0])
                desc.append(i[1])
        
        if (len(i) == 1):
            if i[0].isnumeric() == True:
                val.append(i[0])                
            else:
                tax_ty.append(i[0])
        
    # This is necesary becuase the tax code of the sell item is not always present 
    
    for i in val:
        if len(val) > len(tax_ty):
            tax_ty.append('0')

    create = np.arange(len(desc))
    
    # Return the list of items with the necessary atributes
    
    for i in create:
        a = [sku[i],desc[i],val[i],tax_ty[i]]
        items.append(a)

    return items

## Function to extrac the items of one .json

This function only works for .json associated with VALLE DE LILI shop


In [10]:
'''

 Due to the structure of the file, it was necessary to identify the place 
 where the required information of the sold items appeared. This function 
 is responsible for finding the places where the required information is 
 found and cleans the data for better manipulation.
 
 In this case the data is much more disorganized and requires more cleanup.
 
'''

def items_valle_lili(text_split):
    
    
    
    tex = np.arange(len(text_split))
    
    # Set the integers to use 
    
    start1 = 0
    end1 = 0
    start2 = 0
    end2 = 0
    
    # Set the lists to use
    
    data_3 = []
    items_str = []
    data_4 = []
    tazx = []
    sku = []
    desc = []
    val = []
    tax_ty = []
    tems = []
    
    
    # Determines where the block of interest begins and ends
    
    for i in tex:
        if text_split[i] == 'VENDEDOR ELECTRO':
            start1 = i
        if text_split[i][0] == 'D':
            end1 = i
            break
    
    # Extract the necesary data
    
    for i in np.arange(start1+1,end1,1):
        data_3.append(text_split[i])
        
    
    # Clean and extract the necesary SKU and descriptions of the sell items
    
    for i in data_3:
        if i != 'SUBTOTAL':
            items_str.append(i.split(' ',1))
    

    # Choose the block of data that contain the price and tax code of the sell items
    
    for i in tex:
        if (':' in text_split[i]):
            start2 = i
        if ('RESOL.' in text_split[i]):
            end2 = i
            break
    
    # Clean and extract the necesary data
    
    for i in tex:
        if (end2 > i > start2 and ('-' not in text_split[i])):
            data_4.append(text_split[i])
            if ('-' in text_split[i+1]):
                break
    
    # This is necessary to remove unnecessary data
    
    data_4.pop()
    
    # This is necessary to remove unnecessary data
    
    for i in np.arange(len(data_4)):
        if (data_4[i+1] != data_4[i]):
            data_4.remove(data_4[i+1])
            break
            
    # This is necessary to remove unnecessary data
    
    for i in np.arange(len(data_4)):
        if (data_4[i].isalpha() == True):
            data_4.remove(data_4[i])
            break
    
    # Store the necessary data
    
    for i in data_4:
        tazx.append(i.split(' ',1))
    
    # Store the SKU and descriptions data
    
    for i in np.arange(len(items_str)):
        sku.append(items_str[i][0])
        desc.append(items_str[i][1])
        
    # Store the price and tax code data
    
    for i in tazx:
        if (len(i) == 2):
            if (len(i[0]) <= 8 and i[0].isnumeric() == True):
                val.append(i[0])
                tax_ty.append(i[1])
            else: 
                sku.append(desc[i])
        
        if (len(i) == 1):
            if i[0].isnumeric() == True:
                val.append(i[0])
                tax_ty.append('0')
            else:
                tax_ty.append(i[0])
    
    create = np.arange(len(desc))
    
    # Return the list of items with the necessary atributes

    for i in create:
        a = [sku[i],desc[i],val[i],tax_ty[i]]
        tems.append(a)
        
    return tems

## Function to extract the total & subtotal values of .json file

This function only works for .json associated with CALLE 80 shop 

In [11]:
'''

This function extracts the total and subtotal values from the sales ticket
of CALLE 80 shop

'''


def tot_and_sub_tot(split):
    
    total = np.arange(len(split))
    tot = 0.0
    sub_tot = 0.0

    # Determines the position of the values
    
    for i in total:
        if split[i] == 'TOTAL':
            tot = float(split[i+1])
            sub_tot = float(split[i+1])+float(split[i+2])
            
    return [tot,sub_tot]


## Function to extract the total & subtotal values of .json file

This function only works for .json associated with VALLE DE LILI shop


In [12]:
'''

This function extracts the total and subtotal values from the sales ticket
of VALLE de LILI shop

'''

def tot_and_sub_tot_1(split):
    
    total = np.arange(len(split))
    tot = 0.0
    sub_tot = 0.0
    
    # Determines the position of the values
    
    for i in total:
        if split[i] == 'TOTAL':
            tot = float(split[i-2])-float(split[i-1])
            sub_tot = float(split[i-2])
    
    return [tot,sub_tot]


## Function to extrac the date of OCR .json file

In [13]:
'''

This function determines the date you made the purchase. Works for both stores.

'''

def date(text_split):
    date = []
    lent = np.arange(len(text_split))
    
    # Determines the position where the data is located
    
    for i in lent:
        if ('/' in text_split[i] and (len(text_split[i]) == 10 and text_split[i][0].isalpha() != True )):
            date.append(text_split[i])
    
    return date

## Function to extract the invoice number of the OCR .json file

In [14]:
'''

This function determines the invoice number the purchase ticket. Works for both stores.

'''

def get_ticket(json):
    
    leng = np.arange(len(json['pages'][0]['textAnnotations']))

    ticket = []

    for i in leng:
        if json['pages'][0]['textAnnotations'][i]['description'] == 'TIQUETE':
            ticket.append(json['pages'][0]['textAnnotations'][i]['description']+\
                          ' '+json['pages'][0]['textAnnotations'][i+1]['description']\
                          +' '+json['pages'][0]['textAnnotations'][i+2]['description'])
    
    return ticket
    

## Function to extract the requested information for JUMBO CALLE 80 shop

In [15]:
def get_info_calle80(ticket_json):
    
    # In this line we indentify the hole extract text of the image and create a list for search the info
    
    text = ticket_json['pages'][0]['textAnnotations'][0]['description']
    text_split = text.split('\n')
    
    
    tot,sub = tot_and_sub_tot(text_split)
       
    shop_adress = adress(text_split)
        
    date_ticktet = date(text_split)
    
    ticket_number = get_ticket(ticket_json)
    
    items_ticket = items_calle80(text_split)
    
    # Here the information is organized
    
    information =  {
        'Date': date_ticktet[0],
        'Store adress': shop_adress[0], 
        'Invocice number': ticket_number[0], 
        'Subtotal': sub, 'Total': tot,
        'Items': items_ticket
    }
    
    # The information is presented in JSON format 
    
    information_json = json.dumps(information)
    
    return (information_json)

## Function to extract the requested information for JUMBO VALLE de LILI shop

In [16]:
def get_info_vallelili(ticket_json):
    
    # In this line we indentify the hole extract text of the image and create a list for search the info
    text = ticket_json['pages'][0]['textAnnotations'][0]['description']
    text_split = text.split('\n')
    
    
    tot,sub = tot_and_sub_tot_1(text_split)
    
    shop_adress = adress(text_split)
    
    date_ticktet = date(text_split)

    ticket_number = get_ticket(ticket_json)
    
    items_ticket = items_valle_lili(text_split)
       
    # Here the information is organized
    
    information =  {
        'Date': date_ticktet[0],
        'Store adress': shop_adress[0], 
        'Invocice number': ticket_number[0], 
        'Subtotal': sub, 'Total': tot, 
        'Items':items_ticket
    }
    
    # The information is presented in JSON format 
    
    information_json = json.dumps(information)
    
    
    return(information_json)

In [17]:
get_info_calle80(ticket1)

'{"Date": "10/12/2021", "Store adress": "AVENIDA CALLE 80 No 69Q- 50 TEL:6387000", "Invocice number": "TIQUETE J122 249302", "Subtotal": 86160.0, "Total": 77733.0, "Items": [["8410270241140", "Aceite oliva SU", "27990", "N"], ["7707322030489", "Quinua QUINOACL", "7990", "A"], ["7707322030489", "Quinua QUINOACL", "7990", "A"], ["8004690751060", "Cous cous LA MO", "8990", "N"], ["7702247011056", "YOGURT LIQUIDO", "2650", "A"], ["8410971033785", "Aceitunas EXCEL", "3490", "N"], ["7707298470074", "Tallarines BEST", "8790", "A"], ["7702085003497", "Quinua molida D", "8690", "A"], ["7705326077837", "Tortillinas BIM", "5990", "N"], ["7702253800002", "Maiz pira TOT-R", "3590", "0"]]}'

In [18]:
get_info_vallelili(ticket2)


'{"Date": "04/11/2021", "Store adress": "CARRERA 98 No 16-50", "Invocice number": "TIQUETE J212 341304", "Subtotal": 174196.0, "Total": 151731.0, "Items": [["7702406000150", "Azucar refinado", "8590", "A"], ["7702406000150", "Azucar refinado", "8590", "A"], ["7702406000150", "Azucar refinado", "8590", "A"], ["7702406000150", "Azucar refinado", "8590", "A"], ["7702020212052", "DONAREPA PERLAD", "3690", "A"], ["7702020212052", "DONAREPA PERLAD", "3690", "A"], ["7705491102020", "PANELA EXTRA RE", "4390", "0"], ["7705491102020", "PANELA EXTRA RE", "4390", "0"], ["7702026020507", "Servilleta FAMI", "6590", "N"], ["7702010225123", "FABULOSO LAVAND", "6590", "F"], ["7701018005089", "ACEITE OLEOCALI", "48790", "N"], ["7500435126823", "Shampoo H&S lim", "27990", "N"], ["7622201772840", "Gelatina ROYAL", "3890", "N"], ["7509546672557", "Desodorante SPE", "33990", "N"], ["7702026193003", "Papel hig. FAMIL", "28990", "N"]]}'